In [126]:
import pandas as pd
import numpy as np
import requests
from googleAPIconfig import apiKey
import json
from datetime import datetime

In [303]:
target_city = "Edison, NJ"

target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=33.9829, -118.3338&key=%s" % (apiKey)

target_url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (target_city, apiKey)
 
#latlng = "33.9829, -118.3338"
#latlngStr = latlng.strip("(").strip(")")
#target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=%s&key=%s" % (latlngStr,apiKey)
#geo_data = requests.get(target_url).json()

def getZipCode(latlng):
    print(latlng)
    latlngStr = latlng.strip("(").strip(")")
    target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=%s&key=%s" % (latlngStr,apiKey)
    geo_data = requests.get(target_url1).json()
    #print(json.dumps(geo_data['results'][0], sort_keys=True, indent=4))
    try:
        #print([item['long_name'] for item in geo_data['results'][0]['address_components'] if "postal_code" in item['types']][0])
        return [item['long_name'] for item in geo_data['results'][0]['address_components'] if "postal_code" in item['types']][0]
    except:
        print(geo_data)
        return ""
    
def getDateTime(date_string):
    #print(date_string)
    date_list = [date_string.split("/")[2],date_string.split("/")[0],date_string.split("/")[1]]
    date_list = [np.int(item) for item in date_list]
    return datetime(date_list[0],date_list[1],date_list[2])

#Create a method that cleans the data
def checkResults(iteminSeries):
    if (type(iteminSeries)==str) & (iteminSeries != np.nan):
        return "True"
    else:
        #print(iteminSeries)
        return "False"

In [130]:
# Read csv file
df = pd.read_csv("Crime_Data_from_2010_to_Present.csv")

#Use the method to create a new column that return True/False strings to clean the data
for i,rows in df.iterrows():
    df.at[i,'Location Good'] = checkResults(df.at[i,'Location '])

#Get the subset data in order to have zipCode entries for lat/lng coordinates
df2 = df[df['Location Good']=="True"]

#Create an column for the datetime of when crime occured
for i,rows in df2.iterrows():
    df2.at[i,'Date Occurred (datetime)'] = getDateTime(df2.at[i,"Date Occurred"])

#Filter for the year 2016,2017,2018
df_2016 = df2[[item.year==2016 for item in df2["Date Occurred (datetime)"]]]
df_2017 = df2[[item.year==2017 for item in df2["Date Occurred (datetime)"]]]
df_2018 = df2[[item.year==2018 for item in df2["Date Occurred (datetime)"]]]


#df2['Zip code'] = df2['Location '].map(getZipCode)



/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [291]:
#down select number of rows
row_values = np.arange(0,len(df_2018),dtype=int)
df_2018_ds = df_2018.iloc[np.random.choice(row_values,size=500)]
df_2018_ds = df_2018_ds.drop_duplicates()


In [304]:
#Get the zipCodes from latitude and longitude and send to a new columns
for i,rows in df_2018_ds.iterrows():
    print(i,len(df_2018_ds.at[i,'Location ']))
    df_2018_ds.at[i,'Zip Code'] = getZipCode(df_2018_ds.at[i,'Location '])

1681401 20
(34.0092, -118.2827)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1682860 20
(34.2012, -118.4989)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1701845 6
(0, 0)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1658821 20
(34.0212, -118.4238)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1675045 19
(34.261, -118.4698)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1691790 20
(34.0929, -118.4307)
{'error_message': 'You have exceeded your daily request quota for this API.', 'results': [], 'status': 'OVER_QUERY_LIMIT'}
1667744 20
(34.0928, -118.3136)
{'error_message': 'You have exceeded your daily reques

In [305]:
df_2018_ds.groupby("Zip Code").count().sort_values("DR Number",ascending=False)

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location,Location Good,Date Occurred (datetime)
Zip Code,,,,,,,,,,,,,,,,,,,,,
,495,495,495,495,495,495,495,495,495,433,...,495,495,42,2,0,495,97,495,495,495
90006,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,0,1,1,1
91405,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,0,1,1,1
